# Моделирование роста монокристаллического твердого раствора $Al_xGa_{1-x}N$ методом хлоридной эпитаксии


## Сазанович Владислав М3339

In [40]:
import numpy as np
import pandas as pd
import math

In [41]:
# Для тестов
def is_close(a, b, rel_tol=1e-06, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

In [42]:
row_data = pd.read_csv('q.csv', sep=';', index_col=False, names=["", "Phase", "T1", "T2", "H", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "mu", "sigma", "epsil", "Comment"])
row_data.set_index('', inplace = True)
row_data

,Phase,T1,T2,H,f1,f2,f3,f4,f5,f6,f7,mu,sigma,epsil,Comment
,,,,,,,,,,,,,,,
AlCl,g,298.15,3000.00,-51032.0,318.9948,36.946260,-0.001226,1.188174,5.638541,-5.066135,5.219347,62.4345,3.580,932.00,"1,2"
AlCl2,g,298.15,3000.00,-259000.0,427.2137,56.564090,-0.002961,1.893842,12.400720,-22.654410,21.298980,97.8875,5.300,825.00,"1,2,55"
AlCl3,g,298.15,3000.00,-584100.0,511.8114,81.150420,-0.004835,2.752097,13.400780,-21.280010,16.928680,133.3405,5.130,472.00,"1,2"
GaCl,g,298.15,3000.00,-70553.0,332.2718,37.110520,-0.000746,1.160651,4.891346,-4.467591,5.506236,105.1730,3.696,348.20,1
GaCl2,g,298.15,3000.00,-241238.0,443.2976,57.745845,-0.002265,1.875554,3.661860,-9.356338,15.882450,140.6260,4.293,465.00,1
GaCl3,g,298.15,3000.00,-431573.0,526.8113,82.033550,-0.003486,2.685592,8.278878,-14.567800,12.889900,176.0800,5.034,548.24,1
NH3,g,298.15,3000.00,-45940.0,231.1183,20.522220,0.000716,0.767724,244.629600,-251.690000,146.694700,17.0310,3.000,300.00,"26,1"
H2,g,298.15,3000.00,0.0,205.5368,29.504870,0.000168,0.860656,-14.953120,78.189550,-82.789810,2.0160,2.930,34.10,"4,1"
HCl,g,298.15,3000.00,-92310.0,243.9878,23.159840,0.001820,0.614738,51.166040,-36.895020,9.174252,36.4610,2.737,167.10,1


Так можно брать данные из строчки по имени:

In [43]:
print(row_data.loc['AlCl'].f1)

318.9948


In [44]:
# Constants
# TODO проверить такая ли константа для всех формул
R = 8.314 # Дж/(кмоль*К)

# Распределение веществ по твердым, жидким и газообазным
solid = {'Al', 'AlN', 'GaN'}
liquid = {'Ga'}
gas = {'AlCl', 'AlCl2', 'AlCl3', 'GaCl', 'GaCl2', 'GaCl3', 'NH3', 'H2', 'HCl', 'N2'}

In [45]:
""" Коэффициенты диффузии (м2/сек) для газообразных компонент
ind - 'AlCl', 'GaN, ... 
t - температура в К
p - полное давление в системе
"""
def calcDGas(ind, t, p):
    assert (ind in gas), "This function could caclulate D only for gaseous substance"
    
    numerator = 2.628 * 10**(-2) * t**(3/2)
    
    d_ind = row_data.loc[ind]
    d_N2 = row_data.loc['N2']
    
    sigma = (d_ind.sigma + d_N2.sigma) / 2
    epsil = (d_ind.epsil * d_N2.epsil)**(1/2)
    mu = (2 * d_ind.mu * d_N2.mu) / (d_ind.mu + d_N2.mu)
    omega = 1.074 * (t / epsil)**(-0.1604)
    
    return numerator / (p * omega * mu**(1/2))


"""
Φ для расчета энергии Гиббса
ind - 'AlCl', 'GaN, ... 
t - температура в К
"""
def calcPhi(ind, t):
    data = row_data.loc[ind]
    x = t / 10**4
    
    return data.f1 + data.f2 * math.log(x) + data.f3 / x**2 + data.f4 / x + data.f5 * x + data.f6 * x**2 + data.f7 * x**3
   
    
"""
Энергия Гиббса
ind - 'AlCl', 'GaN, ... 
t - температура в К
"""
def calcG(ind, t):
    data = row_data.loc[ind]
    return data.H - calcPhi(ind, t) * t

Pa = 100000 # атмосферное давление


def calcK1(t):
    deltaG1 = 2 * calcG('Al', t) + 2 * calcG('HCl', t) - 2 * calcG('AlCl', t) - calcG('H2', t)
    return math.exp(-deltaG1 / (R * t)) / Pa


def calcK2(t):
    deltaG2 = calcG('Al', t) + 2 * calcG('HCl', t) - calcG('AlCl2', t) - calcG('H2', t)
    return math.exp(-deltaG2 / (R * t))


def calcK3(t):
    deltaG3 = calcG('Al', t) - 2 * calcG('AlCl3', t) - 3 * calcG('H2', t) + 6 * calcG('HCl', t)
    return math.exp(-deltaG3 / (R * t)) * Pa

# Первое задание

In [46]:
"""
Парциальные давления компонент вне диффузионного пограничного слоя (Па)
PS Даны в условии поэтому дефайним только для этого задания
ind - 'AlCl', 'GaN, ... 
"""
def calcPg(ind):
    if ind == 'AlCl' or ind == 'AlCl2' or ind == 'AlCl3' or ind == 'H2':
        return 0
    if ind == 'HCl':
        return 10000
    if ind == 'N2':
        return 90000
    raise Exception('Unexpected substance: ' + ind)


def first_task():
    P = 100000 # Па
    print(calcD('AlCl', 400, P))
    
# просто проверить что работает
first_task()
calcK1(100)
calcK2(100)
calcK3(100)
calcPg('AlCl')
calcPg('HCl')
calcPg('N2')

0.00033771956183


4.803098490331315e-309